In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
kingfa_datasets_path = "./kingfadatasets/"

In [3]:
X = np.array(pd.read_csv(kingfa_datasets_path+"data.csv", index_col=0))

In [4]:
y = np.array(pd.read_csv(kingfa_datasets_path+"target.csv", index_col=0))

In [5]:
print(X.shape)
print(y.shape)

(3000, 4096)
(3000, 1)


In [6]:
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1024)

In [8]:
X_train = X_train.reshape(X_train.shape[0], 4096, 1)
X_test = X_test.reshape(X_test.shape[0], 4096, 1)
y_train = keras.utils.to_categorical(y_train, 3)
y_test = keras.utils.to_categorical(y_test, 3)

In [16]:
model = Sequential()

In [17]:
model.add(layers.Conv1D(filters=16, kernel_size=8, input_shape=(4096, 1), data_format="channels_last"))
model.add(layers.Activation('sigmoid'))
model.add(layers.MaxPooling1D(pool_size=2, padding='same'))

In [18]:
model.add(layers.Conv1D(filters=32, kernel_size=6, padding='same'))
model.add(layers.Activation('sigmoid'))
# model.add(layers.MaxPooling1D(pool_size=2, padding='same'))

In [19]:
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5))
# model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('sigmoid'))
model.add(layers.Dense(3))
model.add(layers.Activation('softmax'))

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 4089, 16)          144       
_________________________________________________________________
activation_3 (Activation)    (None, 4089, 16)          0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 2045, 16)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 2045, 32)          3104      
_________________________________________________________________
activation_4 (Activation)    (None, 2045, 32)          0         
_________________________________________________________________
gru_4 (GRU)                  (None, 32)                6240      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
__________

In [21]:
model.compile(optimizer=RMSprop(), loss='squared_hinge', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=300, validation_split=0.1)

Train on 2295 samples, validate on 255 samples
Epoch 1/300
2295/2295 [==============================] - 44s 19ms/step - loss: 0.8167 - acc: 0.3325 - val_loss: 0.8147 - val_acc: 0.3529
Epoch 2/300
2295/2295 [==============================] - 43s 19ms/step - loss: 0.8156 - acc: 0.3190 - val_loss: 0.8160 - val_acc: 0.3255
Epoch 3/300
1856/2295 [=======================>......] - ETA: 8s - loss: 0.8152 - acc: 0.3341

In [ ]:
score = model.evaluate(X_test, y_test)
print("LOSS:", score[0])
print("Accuracy:", score[1])

In [ ]:
model.predict(X_test)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()